# Assignment 3

# Question 1

```
INPUT SENTENCE 1 I love apples
VADER OUTPUT {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}

```
 There is a correct interpretation that the sentence is positive, given by the fact that it contains the word 'love', which in the lexicon has a sentiment rating of 3.2 (close to 4), which indicates a positive sentiment.

```
INPUT SENTENCE 2 I don't love apples
VADER OUTPUT {'neg': 0.627, 'neu': 0.373, 'pos': 0.0, 'compound': -0.5216}
```
 There is a correct interpretation that the sentence is a little negative and neutral, given by the fact that it contains the word 'love', which in the lexicon has a sentiment rating of 3.2 (close to 4), but also has the negation "don't". This negation inverts the polarity of "love", that is, although "love" is a positive word, it is interpreted as negative one. 

```
INPUT SENTENCE 3 I love apples :-)
VADER OUTPUT {'neg': 0.0, 'neu': 0.133, 'pos': 0.867, 'compound': 0.7579}
```
 There is a correct interpretation that it is highly positive. It recognizes the word "love" as highly positive in the lexicon and also considers the presence of the emoticon ":-)" as an intensifier, contributing to the overall positive sentiment.
```

INPUT SENTENCE 4 These houses are ruins
VADER OUTPUT {'neg': 0.492, 'neu': 0.508, 'pos': 0.0, 'compound': -0.4404}

```
 Here, VADER is a little unsure wether the sentence is negative or neutral, although not positive, which is good. That is because "ruins" is considered in the lexicon as negative (-1.9), but as there are no other negative intensifiers, it is more neutral.
```
INPUT SENTENCE 5 These houses are certainly not considered ruins
VADER OUTPUT {'neg': 0.0, 'neu': 0.51, 'pos': 0.49, 'compound': 0.5867}
```
 Here, VADER gives a correct interpretation of the sentence as being positive and neutral. That is because, although it recognizes "ruins" as being negative, it turns it to positive because of the intensified negator "certainly not". As there are no more intensifiers of sentiment, the sentence also oscilates torwards a neutral sentiment.

```
INPUT SENTENCE 6 He lies in the chair in the garden
VADER OUTPUT {'neg': 0.286, 'neu': 0.714, 'pos': 0.0, 'compound': -0.4215}

```
 Here, there is a bit of misinterpretation. Although the sentence is completely neutral, given that is is just an information, VADER interprets the word "lies" as negative in its lexicon. That is because this word is a homonym to the infinitive verb "lies", which means "to not tell the truth", which induces a negative sentiment. This makes it interpret the sentence as having 28.6% of negative content, although it should be 100% neutral.

```
INPUT SENTENCE 7 This house is like any house
VADER OUTPUT {'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.3612
```

 Here, VADER made the same mistake as INPUT SENTENCE 6, but instead of negative, it interpreted the sentence as being one third positive, although it should also be completely neutral. That is because it recognizes the word "like" as having a positive valence score in its lexicon, because it is a homonym to the verb "like" as in "to like something".

# Question 2


In [60]:
import json
my_tweets = json.load(open('my_tweets.json'))

In [61]:
for id_, tweet_info in my_tweets.items():
    print(id_,tweet_info)
    break

1 {'sentiment_label': 'negative', 'text_of_tweet': 'I hate when pretty girls act mean cause bitch now you’re ugly.', 'tweet_url': 'https://twitter.com/raveneliseee/status/1763188901617819703'}


# Question 3

In [62]:
def vader_output_to_label(vader_output):
    
    
    """
    map vader output e.g.,
    {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4215}
    to one of the following values:
    a) positive float -> 'positive'
    b) 0.0 -> 'neutral'
    c) negative float -> 'negative'
    
    :param dict vader_output: output dict from vader
    
    :rtype: str
    :return: 'negative' | 'neutral' | 'positive'
    """
    compound = vader_output['compound']
    
    if compound < 0:
        return 'negative'
    elif compound == 0.0:
        return 'neutral'
    elif compound > 0.0:
        return 'positive'
    
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.0}) == 'neutral'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.01}) == 'positive'
assert vader_output_to_label( {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': -0.01}) == 'negative'

In [63]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
nlp = spacy.load('en_core_web_sm')

vader_model = SentimentIntensityAnalyzer()

tweets = []
all_vader_output = []
gold = []

# settings (to change for different experiments)
to_lemmatize = True 
pos = set()

def run_vader(textual_unit, 
              parts_of_speech_to_consider=None, lemmatize=False,
              verbose=0):
    """
    Run VADER on a sentence from spacy
    
    :param str textual unit: a textual unit, e.g., sentence, sentences (one string)
    (by looping over doc.sents)
    :param bool lemmatize: If True, provide lemmas to VADER instead of words
    :param set parts_of_speech_to_consider:
    -None or empty set: all parts of speech are provided
    -non-empty set: only these parts of speech are considered.
    :param int verbose: if set to 1, information is printed
    about input and output
    
    :rtype: dict
    :return: vader output dict
    """
    doc = nlp(textual_unit)
        
    input_to_vader = []

    for sent in doc.sents:
        for token in sent:

            to_add = token.text

            if lemmatize:
                to_add = token.lemma_

                if to_add == '-PRON-': 
                    to_add = token.text

            if parts_of_speech_to_consider:
                if token.pos_ in parts_of_speech_to_consider:
                    input_to_vader.append(to_add) 
            else:
                input_to_vader.append(to_add)

    scores = vader_model.polarity_scores(' '.join(input_to_vader))
    
    if verbose >= 1:
        print()
        print('INPUT SENTENCE', sent)
        print('INPUT TO VADER', input_to_vader)
        print('VADER OUTPUT', scores)

    return scores

for id_, tweet_info in my_tweets.items():
    the_tweet = tweet_info['text_of_tweet']
    vader_output = run_vader(the_tweet,lemmatize=to_lemmatize)# run vader
    vader_label = vader_output_to_label(vader_output)# convert vader output to category
    
    tweets.append(the_tweet)
    all_vader_output.append(vader_label)
    gold.append(tweet_info['sentiment_label'])
    
# use scikit-learn's classification report

### 3.a

In [64]:
total_tweets = len(tweets)
wrong_guesses = {}

for i in range(total_tweets):
    print("tweet:",tweets[i])
    print("vader label:", all_vader_output[i])
    print("gold label:", gold[i])
    if all_vader_output[i] != gold[i]:
        category_tweets_dict = wrong_guesses.get(gold[i],{})
        wrong_tweets_by_category = category_tweets_dict.get(all_vader_output[i],[])
        wrong_tweets_by_category.append(tweets[i])
        category_tweets_dict[all_vader_output[i]] = wrong_tweets_by_category
        wrong_guesses[gold[i]] = category_tweets_dict
        
    print()

tweet: I hate when pretty girls act mean cause bitch now you’re ugly.
vader label: negative
gold label: negative

tweet: I hate when people take kindness for granted
vader label: positive
gold label: negative

tweet: I really hate the fact that some people don’t have the patience to read and understand basic communication & instructions……  it’s unfair to humanity.
vader label: negative
gold label: negative

tweet: i hate feeling like i’m begging someone to do something they said they would. just keep your word please.
vader label: positive
gold label: negative

tweet: People who do things and blame it on alcohol disgust me
vader label: negative
gold label: negative

tweet: People that are love obsessed disgust tf out of me, cause why do you always want to be in love(a relationship) are you possessed?
vader label: negative
gold label: negative

tweet: Your friend is literally shtting on your interest to your face after you expressed discomfort… I doubt theyd care this much about you
vad

In [65]:
from sklearn.metrics import classification_report

report = classification_report(gold,all_vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.65      0.73      0.69        15
     neutral       0.80      0.25      0.38        16
    positive       0.61      0.89      0.72        19

    accuracy                           0.64        50
   macro avg       0.68      0.63      0.60        50
weighted avg       0.68      0.64      0.60        50



#### Having a look at the metrics, this is what we can extract:
* VADER has a good precision (80%) on neutral sentiment data, which means that most of the time it guesses the sentiment as being neutral, it is correct. On the other hand, it has a poor performance when we consider the recall, that is, a large number of sentences which were neutral were either recognized as being positive or negative. This indicates that it has to be trained more on neutral data. This might have happened because the neutral training data is largely composed of scientific news headlines, and sometimes the subject of the article can be either interpreted as negative (such as diseases) or positive (such as health treatment improvements).

* VADER also shows a good recall on positive data. However, the precision is a bit low. This indicates that although it almost does not miss a positive labeled tweet, it assumes a good part of the tweets were positive when in fact they were neutral or negative. This indicates the need to improve training on positive sentiment data.

* Lastly, VADER shows a similar performance in negative data as it did on positive, when it comes to precision. However, the recall is significantly worse, making it clear that there was a good part of tweets which were negative but were not recognized.

#### Although the metrics observed gave hints about what needs improvement, in this classification problem it is unclear wether we need to improve recall or precision. That is because there is no consequence whatsoever of incorectly guessing a category.

### 3.b

### Negative

In [66]:
for category,tweets in wrong_guesses["negative"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print("-",tweet)
print("---")

Tweets guessed as positive:

- I hate when people take kindness for granted
- i hate feeling like i’m begging someone to do something they said they would. just keep your word please.
- Your friend is literally shtting on your interest to your face after you expressed discomfort… I doubt theyd care this much about you
- Nothing worse than a woman who blames her bad behavior on her zodiac sign
---


#### In the sentences that VADER guessed as being negative, they were all incorrectly guessed as positive. In the first one, this could be attributed to the fact that it also contains the word "kindness", which has a really positive valence on the lexicon (2.3), and maybe that could be bringing the score up. Apart from the third sentence, which has the word "friend", all other tweets are guessed really innaccurately. That maybe can be caused by grammatical erros and poor harmony with the lexicon, so that VADER has not much data out of a sentence to correctly guess what sentiment it is.

### Positive

In [67]:
for category,tweets in wrong_guesses["positive"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print("-",tweet)
    print()
print("---")

Tweets guessed as negative:

- if i ever get to hear him sing everythingoes in person, all my discomfort would wash away

Tweets guessed as neutral:

- Gm legends 💜
Now, this guy is the definition of humble. What a legend 👊

---


#### The first sentence is guessed as negative. That can be attributed to the fact that it contains the work "discomfort", which is a negative word according to the lexicon. The other sentence is guessed as neutral, and this can maybe due to the fact that VADER does not recognize "the definition of humble" as a compliment, maybe because "humble" is not in the lexicon, although this sentence contains emojis which could give a hint to the sentence being positive.

### Neutral

In [68]:
for category,tweets in wrong_guesses["neutral"].items():
    print(f"Tweets guessed as {category}:\n")
    for tweet in tweets:
        print("-",tweet)
    print()
print("---")

Tweets guessed as positive:

- Here’s a look at some of the highlights worth watching from late-night TV.
- Be honest, does banana bread taste nice to you?
- Deep brain stimulation (DBS) maps dysfunctional frontal cortical circuits in dystonia, Parkinson's disease, Tourette's syndrome, and OCD, which helped to guide stimulation sites in other patients
@b_hollunder NingfeiL @andreashorn_ @netstim_org
- As impressive as they seem, the latest computer brains usually fail at tasks that require generalizing concepts — tasks that come easily to humans.
- A retrovirus — embedded in the DNA of jawed vertebrates — helps turn on production of a protein needed to insulate nerve fibers. Such insulation, called myelin, may have helped make speedy thoughts and complex brains possible.
- Today, artificial intelligence can identify potential drug targets and develop medicines from scratch. But whether the approach can craft drugs that help patients remains to be seen.
- OpenAI wants to make a walking,

#### We can see that the neutral sentiment is the hardest one for VADER to guess precisely. That can be attributed to the fact that it takes the words inside the headlines (such as "isn't good", "smarter", "impressive") and retrieve their sentiment from the lexicon. This affects the sentence as a whole, as there are not many intensifiers or other attributes which make the sentiment of the sentence clear.

# Question 4

In [69]:
from sklearn.datasets import load_files
import pathlib

In [70]:
cwd = pathlib.Path.cwd()

In [71]:
airline_tweets_folder = cwd.joinpath('airlinetweets')

In [72]:
print('this will print True if the folder exists:', 
      airline_tweets_folder.exists())

this will print True if the folder exists: True


In [73]:
dataset = load_files(airline_tweets_folder)

In [74]:
print(dataset.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


* Run VADER (as it is) on the set of airline tweets 

In [75]:
gold = []
for target in dataset.target:
    gold.append(dataset.target_names[target])

vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=False)
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.80      0.51      0.63      1750
     neutral       0.60      0.51      0.55      1515
    positive       0.56      0.88      0.68      1490

    accuracy                           0.63      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.66      0.63      0.62      4755



* Run VADER on the set of airline tweets after having lemmatized the text

In [76]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=True)
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.79      0.52      0.63      1750
     neutral       0.60      0.49      0.54      1515
    positive       0.56      0.88      0.68      1490

    accuracy                           0.62      4755
   macro avg       0.65      0.63      0.62      4755
weighted avg       0.65      0.62      0.62      4755



* Run VADER on the set of airline tweets with only adjectives

In [77]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=False,parts_of_speech_to_consider=["ADJ"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.87      0.21      0.34      1750
     neutral       0.41      0.89      0.56      1515
    positive       0.66      0.44      0.53      1490

    accuracy                           0.50      4755
   macro avg       0.64      0.52      0.48      4755
weighted avg       0.66      0.50      0.47      4755



* Run VADER on the set of airline tweets with only adjectives and after having lemmatized the text

In [78]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=True,parts_of_speech_to_consider=["ADJ"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.86      0.21      0.34      1750
     neutral       0.41      0.89      0.56      1515
    positive       0.66      0.44      0.53      1490

    accuracy                           0.50      4755
   macro avg       0.64      0.52      0.48      4755
weighted avg       0.65      0.50      0.47      4755



* Run VADER on the set of airline tweets with only nouns

In [79]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=False,parts_of_speech_to_consider=["NOUN"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.73      0.14      0.23      1750
     neutral       0.36      0.82      0.50      1515
    positive       0.54      0.33      0.41      1490

    accuracy                           0.42      4755
   macro avg       0.54      0.43      0.38      4755
weighted avg       0.55      0.42      0.37      4755



* Run VADER on the set of airline tweets with only nouns and after having lemmatized the text

In [80]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=True,parts_of_speech_to_consider=["NOUN"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.71      0.15      0.25      1750
     neutral       0.36      0.82      0.50      1515
    positive       0.53      0.32      0.40      1490

    accuracy                           0.42      4755
   macro avg       0.53      0.43      0.38      4755
weighted avg       0.54      0.42      0.37      4755



* Run VADER on the set of airline tweets with only verbs

In [81]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=False,parts_of_speech_to_consider=["VERB"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.78      0.29      0.42      1750
     neutral       0.38      0.82      0.52      1515
    positive       0.57      0.34      0.43      1490

    accuracy                           0.47      4755
   macro avg       0.58      0.48      0.46      4755
weighted avg       0.59      0.47      0.45      4755



* Run VADER on the set of airline tweets with only verbs and after having lemmatized the text

In [82]:
vader_output = []

for tweet in dataset.data:
    scores = run_vader(str(tweet),lemmatize=True,parts_of_speech_to_consider=["VERB"])
    vader_output.append(vader_output_to_label(scores))

report = classification_report(gold,vader_output)
print(report)

              precision    recall  f1-score   support

    negative       0.74      0.29      0.42      1750
     neutral       0.38      0.79      0.51      1515
    positive       0.57      0.35      0.43      1490

    accuracy                           0.47      4755
   macro avg       0.56      0.48      0.45      4755
weighted avg       0.57      0.47      0.45      4755



#### It is possible to note that:

* Lemmatization does not have a significant impact on the sentiment evaluation given by VADER. Sometimes, the scores go up or down a little bit, but in general the scores before and after lemmatization are quite similar.

* Adjectives are the most important part of speech for sentiment analysis. That can be justified given that adjectives are generally responsible for giving the sentiment to a sentence. Take, for example, the phrase "Something is good". Here, "something" is a noun, "is" a verb, and "good" an adjective. Out of those 3 words, the only one which gives information on sentiment is "good", which is a positive adjective, while the others are neutral. For instance, if we change the adjective "good" to "bad", the sentiment of the phrase changes. 

* Although ajectives are the most important parts of speech, VADER's analysis performs better if we don't filter out other parts of speech (the first 2 experiments gave out the best results)

## Question 5

In [83]:
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=42)

configurations = [
    {'vectorizer': TfidfVectorizer(min_df=2,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'TF-IDF, min_df=2'},
    {'vectorizer': TfidfVectorizer(min_df=5,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'TF-IDF, min_df=5'},
    {'vectorizer': TfidfVectorizer(min_df=10,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'TF-IDF, min_df=10'},
    {'vectorizer': CountVectorizer(min_df=2,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'Bag of Words, min_df=2'},
    {'vectorizer': CountVectorizer(min_df=5,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'Bag of Words, min_df=5'},
    {'vectorizer': CountVectorizer(min_df=10,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english')), 'name': 'Bag of Words, min_df=10'},
]

for config in configurations:
    vectorizer = config['vectorizer']
    
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    clf = MultinomialNB().fit(X_train_vec, y_train)
    
    y_pred = clf.predict(X_test_vec)
    
    print(f"Classification Report for {config['name']}:")
    print(classification_report(y_test, y_pred,target_names=dataset.target_names))
    print("-" * 80)


/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Classification Report for TF-IDF, min_df=2:
              precision    recall  f1-score   support

    negative       0.82      0.93      0.87       335
     neutral       0.85      0.73      0.78       305
    positive       0.83      0.83      0.83       311

    accuracy                           0.83       951
   macro avg       0.83      0.83      0.83       951
weighted avg       0.83      0.83      0.83       951

--------------------------------------------------------------------------------


/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Classification Report for TF-IDF, min_df=5:
              precision    recall  f1-score   support

    negative       0.83      0.90      0.86       335
     neutral       0.80      0.74      0.77       305
    positive       0.84      0.82      0.83       311

    accuracy                           0.82       951
   macro avg       0.82      0.82      0.82       951
weighted avg       0.82      0.82      0.82       951

--------------------------------------------------------------------------------


/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


...

Classification Report for Bag of Words, min_df=2:
              precision    recall  f1-score   support

    negative       0.83      0.93      0.88       335
     neutral       0.85      0.76      0.80       305
    positive       0.86      0.85      0.85       311

    accuracy                           0.85       951
   macro avg       0.85      0.84      0.84       951
weighted avg       0.85      0.85      0.85       951

--------------------------------------------------------------------------------


/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Classification Report for Bag of Words, min_df=5:
              precision    recall  f1-score   support

    negative       0.84      0.93      0.88       335
     neutral       0.84      0.76      0.80       305
    positive       0.86      0.84      0.85       311

    accuracy                           0.85       951
   macro avg       0.85      0.84      0.84       951
weighted avg       0.85      0.85      0.84       951

--------------------------------------------------------------------------------


/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Classification Report for Bag of Words, min_df=10:
              precision    recall  f1-score   support

    negative       0.83      0.93      0.88       335
     neutral       0.84      0.77      0.81       305
    positive       0.86      0.82      0.84       311

    accuracy                           0.85       951
   macro avg       0.85      0.84      0.84       951
weighted avg       0.85      0.85      0.84       951

--------------------------------------------------------------------------------


- a. Given above
- b. The accuracy, precision and recall for all categories were pretty high for most configurations, with differences under 5%. Bag of Words method shows a slightly better performance. 
    - Best Performing: The negative category shows a high precision and recall consistently for all settings.
    - The frequency changing from "2" to "5" to "10" has no significant effect on the accuracy or recall. Lowering the limit on what frequency of words to include do not add much to the score. This means a core vocabulary of frequent, discriminative terms is sufficient for accurate classification.

 
 # Question 6

In [84]:
def important_features_per_class(vectorizer,classifier,n=80):
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names_out() #had to edit the function to get it to work
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
#important_features_per_class(airline_vec, clf)

In [85]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.2, random_state=42)
vectorizer = CountVectorizer(min_df=2,tokenizer=nltk.word_tokenize,stop_words=stopwords.words('english'))

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

clf = MultinomialNB().fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

important_features_per_class(vectorizer, clf)

/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/temp/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


Important words in negative documents
0 1518.0 @
0 1393.0 united
0 1246.0 .
0 419.0 ``
0 403.0 flight
0 380.0 ?
0 362.0 !
0 332.0 #
0 207.0 n't
0 150.0 ''
0 126.0 's
0 119.0 service
0 117.0 :
0 106.0 virginamerica
0 105.0 get
0 95.0 customer
0 95.0 cancelled
0 86.0 delayed
0 86.0 bag
0 85.0 plane
0 78.0 time
0 77.0 ;
0 73.0 http
0 73.0 -
0 71.0 hours
0 71.0 &
0 69.0 ...
0 69.0 'm
0 65.0 gate
0 64.0 airline
0 62.0 amp
0 61.0 still
0 59.0 late
0 58.0 would
0 58.0 hour
0 58.0 help
0 58.0 2
0 55.0 worst
0 54.0 flights
0 53.0 ca
0 52.0 waiting
0 50.0 like
0 50.0 delay
0 49.0 one
0 48.0 never
0 45.0 us
0 45.0 flightled
0 45.0 (
0 45.0 've
0 44.0 back
0 44.0 $
0 43.0 3
0 42.0 )
0 41.0 ever
0 40.0 really
0 40.0 lost
0 39.0 thanks
0 39.0 fly
0 39.0 due
0 38.0 luggage
0 37.0 wait
0 36.0 u
0 36.0 seat
0 36.0 day
0 34.0 trying
0 34.0 people
0 34.0 hold
0 34.0 crew
0 33.0 last
0 32.0 baggage
0 32.0 another
0 31.0 check
0 31.0 bags
0 31.0 4
0 30.0 ticket
0 30.0 experience
0 30.0 could
0 30.0 airport

- a. Given Above
- b.
    - For negative I assumed alot of words from complaints about people who have flown and had a bad experience. ("delayed","late","dirty") Nuetral would likely have words from news articles about the finances or the company. ("jetblue", "time", "travel") Positive would have words from good reviews, and customer experiences, and also announcements from airlines' twitter account. ("awesome", "friendly")
    
    - I did not expect the punctuation. The "@" is understandable because the tweets likely have people mentioning others. I also saw "http" somewhere. I think perhaps the stopwords function is not working. 

    - Perhaps the punctuation should be removed, the content words and airline names can also be removed, as they do not hold sentimental value. 